# Quickstart MNIST demo <img src="https://dutchanalytics.com/wp-content/uploads/2020/02/svg_logo_dutch_analytics-1.svg" width="50">
Deploy your first model in Xenia using the python client libraries.


The first step is to download a prepared model and some sample data

In [ ]:
!curl -X GET https://storage.googleapis.com/da-xenia/example-model-package/mnist_model_package.zip -o mnist_model_package.zip
!curl -X GET https://storage.googleapis.com/da-xenia/example-model-package/1.jpg -o 1.jpg

Add your API token, provide a project name, model name and model version name. 
Afterward initialize the client library.

In [ ]:
API_TOKEN='Token <YOUR API_KEY>'
PROJECT_NAME='<YOUR PROJECT NAME>'
MODEL_NAME='<YOUR MODEL NAME>'
MODEL_VERSION='<YOUR MODEL VERSION>'

import xenia_python_client_library as xenia
configuration = xenia.Configuration()
configuration.api_key['Authorization'] = API_TOKEN
client = xenia.ApiClient(configuration)
api = xenia.CoreApi(client)

## Deploy the model
Create a model for mnist.

In [ ]:
mnist_model_template = xenia.ModelCreate(
    name=f'{MODEL_NAME}',
    description='A model to classify handwritten digits.',
    input_type='structured',
    output_type='structured',
    input_fields=[
        xenia.ModelInputFieldCreate(
            name='input_blob',
            data_type='blob',
        )
    ],
    output_fields=[
        xenia.ModelOutputFieldCreate(
            name='prediction',
            data_type='int',
        ),
        xenia.ModelOutputFieldCreate(
            name='probability',
            data_type='double'
        )
    ]
)

mnist_model = api.models_create(
    project_name=f'{PROJECT_NAME}',
    data=mnist_model_template
)
print(mnist_model)

Create a model version for mnist

In [ ]:
mnist_model_version_template = xenia.ModelVersionCreate(
    version=f'{MODEL_VERSION}',
    language='python3.6',
    memory_allocation=1024,
    maximum_instances=1,
    minimum_instances=0
)

mnist_model_version = api.model_versions_create(
    project_name=f'{PROJECT_NAME}',
    model_name=f'{MODEL_NAME}',
    data=mnist_model_version_template
)
print(mnist_model_version)

Upload the prepared model file to the created model version

In [ ]:
file_upload_result =api.model_versions_file_upload(
    project_name=f'{PROJECT_NAME}',
    model_name=f'{MODEL_NAME}',
    version=f'{MODEL_VERSION}',
    file='mnist_model_package.zip'
)

Check if model version is finished building.
This can take a few minutes.

In [ ]:
from time import sleep
status = 'building'
print(status)
while status != 'available':    
    model_version_status = api.model_versions_get(       
        project_name=f'{PROJECT_NAME}',        
        model_name=f'{MODEL_NAME}',        
        version=f'{MODEL_VERSION}'    
    )    
    status = model_version_status.status
    sleep(1)
print(status)

## Creating a request
Do a model request. Make sure the model is in 'available' state before performing model requests.

First upload the input image.

In [ ]:
blob = api.blobs_create(project_name=f'{PROJECT_NAME}', file='1.jpg')

Make a request using the blob id as input.

In [ ]:
data = {'input_blob': blob.id}
model_result = api.model_requests_create(
    project_name=f'{PROJECT_NAME}',
    model_name=f'{MODEL_NAME}',
    version=f'{MODEL_VERSION}',
    data=data
)
print(model_result)

## Cleanup
Cleanup created model

In [ ]:
api.models_delete(
    project_name=f'{PROJECT_NAME}',
    model_name=f'{MODEL_NAME}'
)